In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *

In [3]:
from fastai.collab import *
from fastai.tabular.all import *

In [4]:
path = untar_data(URLs.ML_100k)

In [5]:
Path.BASE_PATH = "/"
path.ls()

(#23) [Path('home/mchristos/.fastai/data/ml-100k/u2.base'),Path('home/mchristos/.fastai/data/ml-100k/u.genre'),Path('home/mchristos/.fastai/data/ml-100k/u.user'),Path('home/mchristos/.fastai/data/ml-100k/u3.test'),Path('home/mchristos/.fastai/data/ml-100k/ub.test'),Path('home/mchristos/.fastai/data/ml-100k/u.occupation'),Path('home/mchristos/.fastai/data/ml-100k/ua.test'),Path('home/mchristos/.fastai/data/ml-100k/u5.base'),Path('home/mchristos/.fastai/data/ml-100k/u4.test'),Path('home/mchristos/.fastai/data/ml-100k/u.info')...]

In [6]:
ratings = pd.read_csv(path/"u.data", delimiter="\t", header=None, names = ["user", "movie", "rating", "timestamp"])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
most_watched_users = ratings.groupby("user").count().sort_values(by="movie", ascending=False).index[:10]
most_watched_movies = ratings.groupby("movie").count().sort_values(by="user", ascending=False).index[:20]
most_watched_users, most_watched_movies
mostwatched = ratings.loc[ratings.user.isin(most_watched_users) & ratings.movie.isin(most_watched_movies)]
# cross user with movie, where the value shown is the rating
# technically there could have been multiple ratings for one user/movie combo, but I know there aint, hence simply choosing the first 
pd.crosstab(mostwatched.user, mostwatched.movie, values=mostwatched.rating, aggfunc=first).fillna("")

movie,1,7,50,56,98,100,117,121,127,172,174,181,222,237,258,286,288,294,300,313
user,,,,,,,,,,,,,,,,,,,,
13,3.0,2.0,5.0,5.0,4.0,5.0,3.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,4.0,3.0,1.0,2.0,1.0,4.0
234,3.0,2.0,4.0,3.0,4.0,4.0,2.0,,4.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0
276,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,,4.0,4.0,4.0,5.0
303,5.0,4.0,5.0,5.0,5.0,5.0,3.0,3.0,5.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,4.0,4.0,1.0,
393,3.0,4.0,5.0,2.0,,1.0,4.0,4.0,,5.0,,4.0,4.0,4.0,4.0,,3.0,4.0,,4.0
405,,,5.0,4.0,4.0,,,,5.0,5.0,5.0,5.0,,,,,5.0,,,4.0
416,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,,3.0,5.0,5.0,5.0,4.0,4.0,5.0
450,4.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,5.0,4.0,5.0,4.0,3.0,5.0,4.0,4.0,3.0,4.0,4.0,5.0
537,2.0,4.0,4.0,5.0,3.0,4.0,2.0,1.0,5.0,3.0,3.0,2.0,2.0,3.0,4.0,3.0,2.0,1.0,1.0,4.0


The empty entries in this table correspond to user/movie combos where the user has not watched (or rated) that movie. We'd like to try to build some predictions for those, based on the entries we already have. 

Idea: take every user, and every movie, and assign to each a random vector (all the same length) of values. You can think of every weight being about a particular kind of movie, or a movie feature, such as "action" or "super old black and white". To decide how much a user likes a movie, we then just take the dot product of the user's vector (representing how much the user likes each feature) and and movie's vector (representing how much like that feature the movie is). At least, we're just pretending that these features mean something like that, but for now they are just randomly initialized vectors! 

There are called "latent factors" since we don't know what they represent in advance, but they converge onto something over training. 

In [8]:
users = ratings.user.unique()
movies = ratings.movie.unique()
n_users = len(users)
n_movies = len(movies)
n_users, n_movies

(943, 1682)

Randomly initialize lantent factors for each user and each movie 

In [9]:
# we can choose a number of latent features in our vector 
# this is a hyperparameter of our model 
n_latent = 5 

user_factors = torch.rand([n_users, n_latent])
movie_factors = torch.rand([n_movies, n_latent])
user_factors, movie_factors

(tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904],
         [0.6009, 0.2566, 0.7936, 0.9408, 0.1332],
         [0.9346, 0.5936, 0.8694, 0.5677, 0.7411],
         ...,
         [0.8406, 0.5511, 0.8015, 0.6288, 0.8188],
         [0.5063, 0.6668, 0.9932, 0.3371, 0.2965],
         [0.8630, 0.6912, 0.0280, 0.6778, 0.0096]]),
 tensor([[0.2285, 0.6957, 0.7634, 0.3871, 0.9232],
         [0.9969, 0.8899, 0.0408, 0.3181, 0.5304],
         [0.9064, 0.4161, 0.6044, 0.7632, 0.2384],
         ...,
         [0.5317, 0.0142, 0.7693, 0.8163, 0.5516],
         [0.4099, 0.2627, 0.5041, 0.5248, 0.6410],
         [0.4788, 0.9740, 0.8223, 0.6129, 0.0561]]))

To compute a prediction for a given user/movie combo, we can perform matrix multiplication in such a way that we get a new matrix of size (n_users, n_movies) where each value is the dot product of the corresponding latent factor vectors

In [10]:
predictions = user_factors @ movie_factors.T
predictions.shape

torch.Size([943, 1682])

Each entry is a user/movie combo, where the value is the predicted rating (random for now)

In [11]:
def get_prediction(user, movie):
    return (user_factors @ movie_factors.T)[users.index(user), movies.index(movie)]

predictions

tensor([[1.8623, 2.2217, 2.2371,  ..., 1.7751, 1.5488, 2.2384],
        [1.4088, 1.2296, 1.8808,  ..., 1.7751, 1.2929, 1.7743],
        [2.1942, 2.0691, 2.2296,  ..., 2.0464, 1.7503, 2.1301],
        ...,
        [2.1867, 1.9954, 2.1508,  ..., 2.0363, 1.7483, 2.0296],
        [1.7421, 1.4032, 1.6647,  ..., 1.4815, 1.2504, 1.9319],
        [0.9708, 1.6972, 1.6063,  ..., 1.0488, 0.9114, 1.5255]])

In [12]:
Embedding

fastai.layers.Embedding

In [13]:
movies = pd.read_csv(path/'u.item', delimiter="|", encoding="latin-1", usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [14]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [15]:
# CollabDataLoaders by default looks for "user" and "item", and "rating" as the label. Instead of using the movie id lets use the title
dls = CollabDataLoaders.from_df(ratings, item_name="title")
dls.show_batch()

,user,title,rating
0,611,Picture Perfect (1997),3
1,680,Swingers (1996),5
2,391,Contact (1997),3
3,303,Nobody's Fool (1994),4
4,342,Big Night (1996),2
5,782,"Letter From Death Row, A (1998)",3
6,334,Ridicule (1996),2
7,57,"Terminator, The (1984)",3
8,311,Amadeus (1984),4
9,7,Bringing Up Baby (1938),4


In [16]:
print(len(dls.classes["user"]), n_users)
print(len(dls.classes["title"]), n_users)

944 943
1665 943


In [17]:
# embed two possible categorical values as 5-D vectors 
e=Embedding(2,5)
# the indices 0 and 1 get (randomly initialized) 5-D vectors. 
# But in such a way that we can train this, i.e. can take the gradient
# its the same as 1-hot encoding the input index, and using a matrix 
e.forward(tensor([0])), e.forward(tensor([1]))

(tensor([[-0.0092,  0.0048,  0.0095,  0.0146,  0.0089]], grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.0115,  0.0099, -0.0136,  0.0171,  0.0027]], grad_fn=<EmbeddingBackward0>))

In [18]:
# a Module is a "layer" in a model 
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
    
    def forward(self, x):
        # x.shape = [n_batch, 2] 
        # i.e. 2 values for the user and the movie index
        users, movies = x[:,0], x[:,1]
        # shape = [n_batch, n_factors]
        u_factors = self.user_factors(users)
        # shape = [n_batch, n_factors]
        m_factors = self.movie_factors(movies)
        return (u_factors * m_factors).sum(dim=1) + self.user_bias(users), self.movie_bias(movies)

In [19]:
x,y=dls.one_batch()
x.shape

torch.Size([64, 2])

In [20]:
n_users = len(dls.classes["user"])
n_movies = len(dls.classes["title"])
n_factors = 50
n_users, n_movies, n_factors

(944, 1665, 50)

In [21]:
model = DotProduct(n_users, n_movies, n_factors)
learn = Learner(dls, model, loss_func=MSELossFlat())

learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time


AttributeError: 'tuple' object has no attribute 'view'

In [22]:
learn = collab_learner(dls, n_factors=50, y_range = (0,5.5), use_nn=True)
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.970595,0.959374,00:23
1,0.940957,0.911069,00:23
2,0.888132,0.873502,00:21
3,0.801787,0.842856,00:22
4,0.744690,0.844523,00:15
